# SLHDA MVP — Practical Demonstration

This notebook provides a hands-on demonstration of the **Scalar-Ledger Hybrid Data Architecture (SLHDA MVP)**. It illustrates how to store numerical data, perform dynamic tensor aliasing, query objects semantically, and retrieve arrays efficiently for HPC-grade workflows.

## 1. Initialization

We begin by importing the core package and initializing the database and interpreter layers.

In [ ]:
from slhda_mvp.store import SLHDAMVP
from slhda_mvp.interpreter import Interpreter
import numpy as np

# Initialize the database
db = SLHDAMVP()
interp = Interpreter(db)

print('SLHDA MVP initialized successfully.')

## 2. Store a 4x4 Simulation Grid

We simulate a 4x4 grid and store it as a scalar block with semantic tags.

In [ ]:
# Create random grid data
grid_data = np.random.rand(16).tolist()

# Store in SLHDA
grid_id = db.store_object(grid_data, shape=[4,4], tags=['simulation','grid'])
grid = interp.retrieve(grid_id)

print('Stored Grid Object ID:', grid_id)
print('Grid (4x4):\n', grid)

## 3. Dynamic Tensor Aliasing

The same scalar block can be reinterpreted as different tensor shapes without copying data.

In [ ]:
# Alias as a flat vector
vector_id = interp.alias(grid_id, new_shape=[16], new_tags=['vector'])
vector = interp.retrieve(vector_id)
print('Vector alias (16,):\n', vector)

# Alias as a 2x2x4 subgrid
subgrid_id = interp.alias(grid_id, new_shape=[2,2,4], new_tags=['subgrid'])
subgrid = interp.retrieve(subgrid_id)
print('Subgrid alias (2x2x4):\n', subgrid)

## 4. Semantic Queries

Objects can be retrieved using semantic tags. Here we query all objects associated with the simulation.

In [ ]:
# Store another object for demonstration
pressure_data = np.random.rand(16).tolist()
pressure_id = db.store_object(pressure_data, shape=[4,4], tags=['pressure','simulation'])

# Query all simulation objects
sim_ids = db.query('simulation')
print('Simulation object IDs:', sim_ids)

## 5. Batch Retrieval

Retrieve multiple objects efficiently using batch queries.

In [ ]:
batch_arrays = interp.batch_query('simulation')
print('Shapes of retrieved arrays:', [arr.shape for arr in batch_arrays])

## 6. Performance Test

Store and retrieve a large scalar array to demonstrate the efficiency of contiguous storage for HPC applications.

In [ ]:
import time

N = 10**6  # 1 million scalars
perf_id = db.store_object(np.random.rand(N).tolist(), shape=[N], tags=['perf'])

start = time.time()
retrieved = interp.retrieve(perf_id)
end = time.time()

print(f'Retrieved 1M scalars in {end-start:.4f} seconds')

### To sum up:

- **SLHDA MVP** allows semantic, zero-copy, multi-scale storage of numerical data.
- Tensor aliasing enables flexible reinterpretation without data duplication.
- Semantic tags and batch queries support structured workflow pipelines.
- Contiguous scalar storage provides HPC-friendly performance for large datasets.